In [66]:
import pandas as pd
from collections import OrderedDict
def get_sports():
    df_sport_latest = pd.read_csv('sports_articles.csv', encoding = "ISO-8859-1")
    df_sport_2022 = pd.read_csv('sports_articles_2022.csv', encoding = "ISO-8859-1")
    return pd.concat([df_sport_latest, df_sport_2022])


In [67]:
def vocab_2_pdset(columns, df):
    df_vocab_select_columns = df.iloc[:, columns]
    vocab_all_values = df_vocab_select_columns.values.ravel()
    return set(vocab_all_values)

def vocab_2_dict(sets):
    assert(len(sets) == 4)
    word_set = sets[0].union(sets[1],sets[2], sets[3])
    df = pd.DataFrame(list(word_set), columns=["Words"])
    df.sort_values(by="Words", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return OrderedDict.fromkeys(word_set)

def get_vocab_dict():
    df_ods_vocab = pd.read_table('ods_fullforms_2020-08-26.csv', header=None)
    df_ddo_vocab = pd.read_table('ddo_fullforms_2020-08-26.csv', header=None)
    df_vocab = pd.read_table('cor1.02.tsv', header=None)
    df_sport_lingo = pd.read_table('sport_lingo.csv', header=None)

    vocab_set = vocab_2_pdset([1,3], df_vocab)
    ods_vocab_set = vocab_2_pdset([0,1], df_ods_vocab)
    ddo_vocab_set = vocab_2_pdset([0,1], df_ddo_vocab)
    sport_lingo_set = vocab_2_pdset([0], df_sport_lingo)

    return vocab_2_dict([vocab_set, ods_vocab_set, ddo_vocab_set, sport_lingo_set])

ordered_dict = get_vocab_dict()


In [68]:
df_sport = get_sports()

In [83]:
import time
def test_lookup_performance():
    word_to_check = "pligtsejrer"
    start_time = time.time()

    for x in range(1000000):
        isin_dict = word_to_check in ordered_dict

    end_time = time.time()  
    assert(end_time - start_time < 1)

test_lookup_performance()

0.03973650932312012


In [70]:
import re


df_sport_text = df_sport.iloc[:, [0,1,2]]

sport_vocab = df_sport_text.values.ravel()

words_arr = []

def replace_digits(word):
    return re.sub(r'\d+', 'X', word)

def remove_specials(word):
    characters_to_remove = [':', "'", '?', ",", "."]
    new_word = word

    for char in characters_to_remove:
        new_word = new_word.replace(char, '')
    return new_word

def contains_non_alphanumeric(word):
    return bool(re.search(r'[^a-zæøåA-ZÆØÅ0-9]', word))

def formatWord(word):
    if any(char.isdigit() for char in word):
        return replace_digits(word)
    
    
for sentences in range(len(sport_vocab)):
    # print(sport_vocab[sentences])
    sentence = sport_vocab[sentences].strip()
    words = sentence.split()
    for word in range(len(words)):
        w = words[word]
        if any(char.isdigit() for char in w):
            w = replace_digits(w)
        if contains_non_alphanumeric(w):
            w = remove_specials(w)
        words_arr.append(w.lower())

words_sport_unique = set(words_arr)
words_sport_unique_list = list(words_sport_unique)
words_sport_lingo = []

# TODO : brug tensorflow Tokenezier til at omdanne ord til tokens
# TODO : søg i alle leksikoner, søg med og uden bindestreg
# TODO : håndter tal ikke i ordbøger eks ( x-x eller x-årig)

for w in range(len(words_sport_unique)):
    isin_dict = words_sport_unique_list[w] in ordered_dict
    if (isin_dict == False):
        words_sport_lingo.append(words_sport_unique_list[w])

print("total unique words:", len(words_sport_unique) )
print("total sports lingo words:", len(words_sport_lingo) )
print("total articles:", len(df_sport) )


total unique words: 1844
total sports lingo words: 363
total articles: 208


In [71]:
words_sport_lingo
file = open('sport_lingo.txt','w')
for item in words_sport_lingo:
	file.write(item+"\n")
file.close()

In [72]:
def format_2_bool(x):
    if type(x) == bool:
        return x
    assert(type(x) == str)
    x_copy = x
    x_copy = x_copy.strip()
    x_copy = x_copy.lower()
    assert(x_copy == "true" or x_copy == "false")
    if x_copy == "true":
        return True
    else:
        return False

In [73]:

df_sport['isResult'] = df_sport['isResult'].apply(lambda x: format_2_bool(x))

df_sport['isResult'][0:10]

results_true = df_sport.loc[df_sport["isResult"] == True]
results_false = df_sport.loc[df_sport["isResult"] == False]

assert(len(results_true) + len(results_false) == len(df_sport))

print(len(results_true))
print(len(results_false))

117
91
